In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# 加载数据
print("正在加载训练数据...")
train_data = pd.read_csv('train.csv')
print("正在加载测试数据...")
test_data = pd.read_csv('test.csv')

# 准备训练数据
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values

# 数据预处理
# 将像素值从[0, 255]归一化到[0, 1]
X_train = X_train / 255.0
X_test = test_data.values / 255.0

# 将训练数据重塑为28x28像素图像
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# 将标签转换为one-hot编码
y_train = to_categorical(y_train, num_classes=10)

# 将数据分为训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# 构建CNN模型
model = Sequential([
    # 第一个卷积层
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    
    # 第二个卷积层
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    # 将特征图展平
    Flatten(),
    
    # 全连接层
    Dense(128, activation='relu'),
    Dropout(0.5),  # 防止过拟合
    
    # 输出层 - 10个数字类别
    Dense(10, activation='softmax')
])

# 编译模型
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 打印模型结构
model.summary()

# 训练模型
print("开始训练模型...")
history = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=10,
    validation_data=(X_val, y_val),
    verbose=1
)

# 在测试集上生成预测
print("生成预测结果...")
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# 创建提交文件
submission = pd.DataFrame({
    "ImageId": range(1, len(predicted_classes) + 1),
    "Label": predicted_classes
})

# 保存提交文件
submission.to_csv('submission.csv', index=False)
print("预测结果已保存到submission.csv")

正在加载训练数据...
正在加载测试数据...


C:\Users\davis\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

开始训练模型...
Epoch 1/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7352 - loss: 0.8248 - val_accuracy: 0.9676 - val_loss: 0.1017
Epoch 2/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9584 - loss: 0.1379 - val_accuracy: 0.9771 - val_loss: 0.0721
Epoch 3/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9720 - loss: 0.0993 - val_accuracy: 0.9826 - val_loss: 0.0557
Epoch 4/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9767 - loss: 0.0762 - val_accuracy: 0.9852 - val_loss: 0.0466
Epoch 5/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9799 - loss: 0.0664 - val_accuracy: 0.9874 - val_loss: 0.0409
Epoch 6/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9851 - loss: 0.0520 - val_accuracy: 0.9874 - val_loss: 0.0437
Epoch 7/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9841 - loss: 0.0505 - val_accuracy: 0.9881 - val_loss: 0.0417
Epoch 8/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9869 - loss: 0.0442 